In [ ]:



from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install pandas_ods_reader -q
from pandas_ods_reader import read_ods

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data_sentiment = read_ods('/content/drive/MyDrive/NLP Project/Sentiment data/hi_3500.ods',1,headers = False)
data_sentiment

,column.0,column.1
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative
...,...,...
9072,RT @aajtak: #Mojo बजरंग बली का सबसे विशालकाय र...,neutral
9073,RT @aajtak: कहां अचानक स्कूल में पड़ा मंत्री क...,neutral
9074,RT @aajtak: #MoJo सुनिए कुलभूषण की दर्दभरी आपब...,neutral
9075,RT @aajtak: लव जेहाद के नाम पर क्या उत्तर प्रद...,neutral


In [ ]:
data_sentiment.columns = ['text','sentiment']
data_sentiment.dropna()

,text,sentiment
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative
...,...,...
9072,RT @aajtak: #Mojo बजरंग बली का सबसे विशालकाय र...,neutral
9073,RT @aajtak: कहां अचानक स्कूल में पड़ा मंत्री क...,neutral
9074,RT @aajtak: #MoJo सुनिए कुलभूषण की दर्दभरी आपब...,neutral
9075,RT @aajtak: लव जेहाद के नाम पर क्या उत्तर प्रद...,neutral


In [ ]:
# Define a function to remove English words, numeric characters, and specific special characters
def clean_hindi_text(text):
    words = nltk.word_tokenize(text)
    cleaned_words = []
    for word in words:
        if not re.match("^[A-Za-z]*$", word) and not bool(re.search(r'[0-9०-९]', word)):
            # Specify special characters to remove (e.g., @, ., ,, #, &)
            word = re.sub(r'[.,@#&:)(]', '', word)
            cleaned_words.append(word)
    return ' '.join(cleaned_words)

In [ ]:
data_sentiment['text'] = data_sentiment['text'].apply(clean_hindi_text)


In [ ]:
# Split the dataset into training and testing sets
X = data_sentiment['text']  # Your Hindi text data
y = data_sentiment['sentiment']  # Sentiment labels (e.g., positive, negative)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Create TF-IDF vectors from the text data
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Initialize and train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)
# Predict sentiment on the test data
y_pred = rf_classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(classification_rep)

Accuracy: 0.69
              precision    recall  f1-score   support

    negative       0.70      0.68      0.69       658
     neutral       0.82      0.60      0.70       535
    positive       0.61      0.76      0.68       623

    accuracy                           0.69      1816
   macro avg       0.71      0.68      0.69      1816
weighted avg       0.70      0.69      0.69      1816



In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Initialize and train a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment on the test data
y_pred_nb = nb_classifier.predict(X_test_tfidf)

# Evaluate the Naive Bayes model
accuracy_nb = accuracy_score(y_test, y_pred_nb)
classification_rep_nb = classification_report(y_test, y_pred_nb)

# Print evaluation metrics for Naive Bayes
print("Multinomial Naive Bayes Classifier:")
print(f"Accuracy: {accuracy_nb:.2f}")
print(classification_rep_nb)


Multinomial Naive Bayes Classifier:
Accuracy: 0.71
              precision    recall  f1-score   support

    negative       0.68      0.73      0.71       658
     neutral       0.84      0.61      0.71       535
    positive       0.66      0.76      0.71       623

    accuracy                           0.71      1816
   macro avg       0.73      0.70      0.71      1816
weighted avg       0.72      0.71      0.71      1816



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize and train a Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment on the test data
y_pred_gb = gb_classifier.predict(X_test_tfidf)

# Evaluate the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
classification_rep_gb = classification_report(y_test, y_pred_gb)

# Print evaluation metrics for Gradient Boosting
print("Gradient Boosting Classifier:")
print(f"Accuracy: {accuracy_gb:.2f}")
print(classification_rep_gb)


Gradient Boosting Classifier:
Accuracy: 0.61
              precision    recall  f1-score   support

    negative       0.61      0.54      0.57       658
     neutral       0.83      0.53      0.64       535
    positive       0.52      0.75      0.62       623

    accuracy                           0.61      1816
   macro avg       0.66      0.61      0.61      1816
weighted avg       0.65      0.61      0.61      1816



In [ ]:
from sklearn.svm import SVC

# Initialize and train a Support Vector Machine classifier for multiclass classification
svm_classifier = SVC(kernel='linear', decision_function_shape='ovr', random_state=42)
svm_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment on the test data
y_pred_svm = svm_classifier.predict(X_test_tfidf)

# Evaluate the SVM model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

# Print evaluation metrics for SVM
print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {accuracy_svm:.2f}")
print(classification_rep_svm)


Support Vector Machine (SVM) Classifier:
Accuracy: 0.72
              precision    recall  f1-score   support

    negative       0.69      0.76      0.72       658
     neutral       0.77      0.71      0.74       535
    positive       0.72      0.70      0.71       623

    accuracy                           0.72      1816
   macro avg       0.73      0.72      0.73      1816
weighted avg       0.73      0.72      0.72      1816



*italicized text*# SVC on Fake news data for sentiment

In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/NLP Project/Data/train.csv',encoding='utf-8-sig')
test = pd.read_csv('/content/drive/MyDrive/NLP Project/Data/test.csv',encoding='utf-8-sig')

In [ ]:
train.drop(['Unnamed: 0','author','id'], inplace = True, axis = 1)
test.drop(['Unnamed: 0','author','id'], inplace = True, axis = 1)

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
hindi_stopwords = [
    'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'इस', 'इसके', 'इसको', 'इसमें', 'इससे',
    'उनका', 'उनकी', 'उनके', 'उनको', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हें', 'उसका',
    'उसकी', 'उसके', 'उसको', 'उसमें', 'उसने', 'उसी', 'उसे', 'उसका', 'करता', 'करते',
    'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'कारण', 'किसी', 'की', 'कुछ', 'के',
    'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन',
    'जिन्होंने', 'जिन्हों', 'जिस', 'जिसे', 'जी', 'जिसके', 'जिसको', 'जिसमें', 'जिसे', 'तक',
    'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'था', 'थे', 'दिया', 'दूसरे',
    'दो', 'द्वारा', 'ने', 'पर', 'पहले', 'पूरा', 'पूरी', 'फिर', 'बनी', 'बहुत', 'बाद',
    'बाला', 'भी', 'मगर', 'मानो', 'मैं', 'मैंने', 'में', 'यदि', 'यह', 'यहाँ', 'यही',
    'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'लिए', 'लिया', 'लेकिन', 'व', 'वगेरह', 'वर्ग',
    'वह', 'वहाँ', 'वही', 'वाले', 'वाली', 'वाला', 'वाले', 'वाली', 'वाला', 'वाले', 'वाली',
    'वाला', 'सबसे', 'सबसे', 'सकता', 'सकते', 'सकती', 'सकती', 'सब', 'सबसे', 'से', 'ही',
    'है', 'हैं', 'हुआ', 'हुई', 'हुए', 'हो', 'होता', 'होते', 'होती', 'होती', 'होते', 'होते',
    'होती', 'होती', 'होती', 'होना', 'होने', 'अंदर', 'अदि', 'अन्य', 'अप', 'अफ', 'अल', 'अलब',
    'अव', 'अवश्य', 'अस', 'आदि', 'आप', 'आपका', 'आपकी', 'आपके', 'आपने', 'आपने', 'आदि', 'आना',
    'इंहोंने', 'इंहें', 'इंहों', 'इतना', 'इतनी', 'इतने', 'इन', 'इनका', 'इनकी', 'इनके', 'इन्ही',
    'इन्होंने', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसी', 'उंहोंने', 'उंहें',
    'उंहों', 'उपर', 'उसके', 'उससे', 'उसी', 'उसे', 'उसी', 'उसे', 'कर', 'करत', 'करता', 'करती',
    'करते', 'करन', 'करना', 'करने', 'किया', 'किये', 'किये', 'किया', 'कुछ', 'कुल', 'के', 'को', 'को'
]


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download("indian")

nltk.download("stopwords")
def remove_hindi_stopwords(text):
    words = nltk.word_tokenize(text)
    cleaned_words = []
    for word in words:
        # Remove specific words
        if word.lower() not in hindi_stopwords:
            # Remove English words and special characters
            if not re.match("^[A-Za-z]*$", word) and not bool(re.search(r'[0-9०-९]', word)):
                # Specify special characters to remove (e.g., @, ., ,, #, &)
                word = re.sub(r'[.,@#&]', '', word)
                cleaned_words.append(word)
    return ' '.join(cleaned_words)



[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Unzipping corpora/indian.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train['text'] = train['text'].apply(remove_hindi_stopwords)
test['text'] = test['text'].apply(remove_hindi_stopwords)

In [ ]:
# Create TF-IDF vectors from the text data
#tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
train_tfidf = tfidf_vectorizer.transform(train['text'])
test_tfidf = tfidf_vectorizer.transform(test['text'])

In [ ]:
sentim_train = svm_classifier.predict(train_tfidf)
sentim_test = svm_classifier.predict(test_tfidf)

In [ ]:
train['sentiment'] = sentim_train
test['sentiment'] = sentim_test

In [ ]:
sentiment_mapping = {"negative": -1, "neutral": 0, "positive": 1}
train['sentiment'] = train['sentiment'].map(sentiment_mapping)
test['sentiment'] = test['sentiment'].map(sentiment_mapping)

In [ ]:
train['sentiment'].value_counts()

-1    1532
 1     319
 0     204
Name: sentiment, dtype: int64

In [ ]:
test['sentiment'].value_counts()

-1    1386
 1     222
 0     199
Name: sentiment, dtype: int64

In [ ]:
train.head()

,title,text,label,sentiment
0,हाउस डिम एड: हमने जेसन चैफेट के ट्वीट करने तक ...,हाउस डिम एड : हमने अक्टूबर डारेल ल्यूकस इसे क...,1,-1
1,"फ्लिन: हिलेरी क्लिंटन, कैंपस की बड़ी महिला - ब...",कभी जीवन उद्देश्य गोल ओर एक सीधी रेखा सिर बजाय...,0,-1
2,सत्य क्यों आपको निकाल सकता है,क्यों सच निकाल अक्टूबर खुफिया विश्लेषकों और र...,1,-1
3,एकल अमेरिकी हवाई हमले में मारे गए 15 नागरिकों ...,वीडियो नागरिकों एकल अमेरिकी हवाई हमले मार जिन...,1,-1
4,ईरानी महिला को व्यभिचार के लिए मौत के घाट उतार...,छाप ईरान रिवोल्यूशनरी गार्ड तलाशी लेने एक ईरान...,1,-1


In [ ]:
train.to_csv('text_label_sentiment.csv')

Fake news detection

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler


In [ ]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB

pipeline_mnb = Pipeline([
    ('text_features', FeatureUnion([
        ('cv_text1', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer())
        ]))
        # ('cv_text2', Pipeline([
        #     ('selector', ItemSelector(key='title')),
        #     ('cv', CountVectorizer()),
        #     ('tfidf', TfidfTransformer())
        # ]))
    ])),
    ('classifier', MultinomialNB())
])

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression

pipeline_lr = Pipeline([
    ('text_features', FeatureUnion([
        ('cv_text1', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer())
        ]))
        # ('cv_text2', Pipeline([
        #     ('selector', ItemSelector(key='title')),
        #     ('cv', CountVectorizer()),
        #     ('tfidf', TfidfTransformer())
        # ]))
    ])),
    ('classifier', LogisticRegression())
])

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC

pipeline_svc = Pipeline([
    ('text_features', FeatureUnion([
        ('cv_text1', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer())
        ]))
        # ('cv_text2', Pipeline([
        #     ('selector', ItemSelector(key='title')),
        #     ('cv', CountVectorizer()),
        #     ('tfidf', TfidfTransformer())
        # ]))
    ])),
    ('classifier', SVC())
])

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
pipeline_rfc = Pipeline([
    ('text_features', FeatureUnion([
        ('cv_text1', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer())
        ]))
        # ('cv_text2', Pipeline([
        #     ('selector', ItemSelector(key='title')),
        #     ('cv', CountVectorizer()),
        #     ('tfidf', TfidfTransformer())
        # ]))
    ])),
    ('classifier', RandomForestClassifier())
])

In [ ]:
# Split the dataset into training and testing sets
X = train['text']
y_sentiment = train['sentiment']
y_authenticity = train['label']

X_train, X_test, y_sentiment_train, y_sentiment_test, y_authenticity_train, y_authenticity_test = train_test_split(
    X, y_sentiment, y_authenticity, test_size=0.2, random_state=42
)

In [ ]:
X_train

841     वैज्ञानिकों कहना कि अंतरिक्ष अजीब संकेत ’ शायद...
915     यू एस अटॉर्नी जनरल जेफ सेशंस और न्याय विभाग गु...
811     मेम्स ब्रेकिंग : हिलेरी लोगों एक बड़ी भीड़ वजह...
1578    संक्षिप्त प्राप्त विदेश मंत्री सर्गेई लावरोव ए...
636     सोशल मीडिया लुभाने और व्यक्तिगत रूप मिलने व्यव...
                              ...                        
1687    राष्ट्रपति चुनाव ऐतिहासिक और अनिश्चित दोनों अम...
1132    डेमोक्रेटिक जॉन ओस्ऑफ  जो उम्मीदवार अटलांटा जि...
1169    बाल्टिमोर - गेरोन्टा डेविस उद्देश्य जूनियर लाइ...
1337    अमेरिकी चुनाव वोट देने ओबामा अवैध प्रवासियों ध...
886     रिपोर्ट : ट्रम्प बुरे अंतत : हालात अंकों बंद ग...
Name: text, Length: 1644, dtype: object

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline_svc.fit(X_train, y_sentiment_train)
svc_sentiment_train = pipeline_svc.predict(X_train)

svc_sentiment_test = pipeline_svc.predict(X_test)

print(f"Accuracy for sentiments with SVC training:{accuracy_score(y_sentiment_train, svc_sentiment_train)}")
print(f"Accuracy for sentiments with SVC test:{accuracy_score(y_sentiment_test, svc_sentiment_test)}\n")

print(f"Classification rep for sentiments with SVC training\n:{classification_report(y_sentiment_train, svc_sentiment_train)}")
print(f"Classification rep for sentiments with SVC test\n:{classification_report(y_sentiment_test, svc_sentiment_test)}")

print()

pipeline_svc.fit(X_train,y_authenticity_train)
svc_authenticity_train = pipeline_svc.predict(X_train)

svc_authenticity_test = pipeline_svc.predict(X_test)

print(f"Accuracy for news with SVC training:{accuracy_score(y_authenticity_train, svc_authenticity_train)}")
print(f"Accuracy for news with SVC test:{accuracy_score(y_authenticity_test, svc_authenticity_test)}\n")

print(f"Classification rep for news with SVC training\n:{classification_report(y_authenticity_train, svc_authenticity_train)}")
print(f"Classification rep for news with SVC test\n:{classification_report(y_authenticity_test, svc_authenticity_test)}")



KeyError: ignored

In [ ]:
pipeline_lr.fit(X_train, y_sentiment_train)
lr_sentiment_train = pipeline_lr.predict(X_train)

lr_sentiment_test = pipeline_lr.predict(X_test)

print(f"Accuracy for sentiments with LR training:{accuracy_score(y_sentiment_train, lr_sentiment_train)}")
print(f"Accuracy for sentiments with LR test:{accuracy_score(y_sentiment_test, lr_sentiment_test)}\n")

print(f"Classification rep for sentiments with LR training\n:{classification_report(y_sentiment_train, lr_sentiment_train)}")
print(f"Classification rep for sentiments with LR test\n:{classification_report(y_sentiment_test, lr_sentiment_test)}")

print()

pipeline_lr.fit(X_train,y_authenticity_train)
lr_authenticity_train = pipeline_lr.predict(X_train)

lr_authenticity_test = pipeline_lr.predict(X_test)

print(f"Accuracy for news with LR training:{accuracy_score(y_authenticity_train, lr_authenticity_train)}")
print(f"Accuracy for news with LR test:{accuracy_score(y_authenticity_test, lr_authenticity_test)}\n")

print(f"Classification rep for news with LR training\n:{classification_report(y_authenticity_train, lr_authenticity_train)}")
print(f"Classification rep for news with LR test\n:{classification_report(y_authenticity_test, lr_authenticity_test)}")



KeyError: ignored

In [ ]:
pipeline_rfc.fit(X_train, y_sentiment_train)
rfc_sentiment_train = pipeline_rfc.predict(X_train)

rfc_sentiment_test = pipeline_rfc.predict(X_test)

print(f"Accuracy for sentiments with RFC training:{accuracy_score(y_sentiment_train, rfc_sentiment_train)}")
print(f"Accuracy for sentiments with RFC test:{accuracy_score(y_sentiment_test, rfc_sentiment_test)}\n")

print(f"Classification rep for sentiments with RFC training\n:{classification_report(y_sentiment_train, rfc_sentiment_train)}")
print(f"Classification rep for sentiments with RFC test\n:{classification_report(y_sentiment_test, rfc_sentiment_test)}")

print()

pipeline_rfc.fit(X_train,y_authenticity_train)
rfc_authenticity_train = pipeline_rfc.predict(X_train)

rfc_authenticity_test = pipeline_rfc.predict(X_test)

print(f"Accuracy for news with RFC training:{accuracy_score(y_authenticity_train, rfc_authenticity_train)}")
print(f"Accuracy for news with RFC test:{accuracy_score(y_authenticity_test, rfc_authenticity_test)}\n")

print(f"Classification rep for news with RFC training\n:{classification_report(y_authenticity_train, rfc_authenticity_train)}")
print(f"Classification rep for news with RFC test\n:{classification_report(y_authenticity_test, rfc_authenticity_test)}")



KeyError: ignored

In [ ]:
pipeline_mnb.fit(X_train, y_sentiment_train)
mnb_sentiment_train = pipeline_mnb.predict(X_train)

mnb_sentiment_test = pipeline_mnb.predict(X_test)

print(f"Accuracy for sentiments with MNB training:{accuracy_score(y_sentiment_train, y_sentiment_train)}")
print(f"Accuracy for sentiments with MNB test:{accuracy_score(y_sentiment_test, mnb_sentiment_test)}\n")

print(f"Classification rep for sentiments with MNB training\n:{classification_report(y_sentiment_train, y_sentiment_train)}")
print(f"Classification rep for sentiments with MNB test\n:{classification_report(y_sentiment_test, mnb_sentiment_test)}")

print()

pipeline_mnb.fit(X_train,y_authenticity_train)
mnb_authenticity_train = pipeline_mnb.predict(X_train)

mnb_authenticity_test = pipeline_mnb.predict(X_test)

print(f"Accuracy for news with MNB training:{accuracy_score(y_authenticity_train, mnb_authenticity_train)}")
print(f"Accuracy for news with MNB test:{accuracy_score(y_authenticity_test, mnb_authenticity_test)}\n")

print(f"Classification rep for news with MNB training\n:{classification_report(y_authenticity_train, mnb_authenticity_train)}")
print(f"Classification rep for news with MNB test\n:{classification_report(y_authenticity_test, mnb_authenticity_test)}")



KeyError: ignored

In [ ]:
# mnb_sentiment_test
#

In [ ]:
X_train

841     वैज्ञानिकों कहना कि अंतरिक्ष अजीब संकेत ’ शायद...
915     यू एस अटॉर्नी जनरल जेफ सेशंस और न्याय विभाग गु...
811     मेम्स ब्रेकिंग : हिलेरी लोगों एक बड़ी भीड़ वजह...
1578    संक्षिप्त प्राप्त विदेश मंत्री सर्गेई लावरोव ए...
636     सोशल मीडिया लुभाने और व्यक्तिगत रूप मिलने व्यव...
                              ...                        
1687    राष्ट्रपति चुनाव ऐतिहासिक और अनिश्चित दोनों अम...
1132    डेमोक्रेटिक जॉन ओस्ऑफ  जो उम्मीदवार अटलांटा जि...
1169    बाल्टिमोर - गेरोन्टा डेविस उद्देश्य जूनियर लाइ...
1337    अमेरिकी चुनाव वोट देने ओबामा अवैध प्रवासियों ध...
886     रिपोर्ट : ट्रम्प बुरे अंतत : हालात अंकों बंद ग...
Name: text, Length: 1644, dtype: object

In [ ]:
inp = input("Enter news:")
inp = remove_hindi_stopwords(inp)
# [inp]: ['ज़ंजीर बदली रही थी समझा रिहाई गयी']


ans = pipeline_rfc.predict([inp])
# ans

In [ ]:
[inp]

['ज़ंजीर बदली रही थी समझा रिहाई गयी']

In [ ]:
X_test

,text,title
1540,रैंड पॉल चेतावनी दी कि मतदान गड़बड़ी मतदाता मत...,"WATCH: UK Reporter RIPS CNN, Calls Them “Clint..."
554,सीरिया अमेरिका नेतृत्व गठबंधन किए गए ग्यारह हव...,सीरिया में 300 नागरिकों की हत्या के लिए एमनेस्...
1451,एक ऐतिहासिक रात थी जिसने अलंकारिक और अलौकिक प्...,दो उम्मीदवार। दो भाषण। क्लिंटन और ट्रम्प से एक...
887,‘ ट्रम्प अनावरण ' डोनाल्ड ट्रम्प प्रेसिडेंशियल...,Con ट्रम्प ने अनावरण किया ’डोनाल्ड ट्रम्प के र...
1167,वाशिंगटन - डोनाल्ड जे। ट्रम्प ताइवान राष्ट्रपत...,ट्रम्प ने बेडकॉक चाइना पॉलिसी का इस्तेमाल बार्...
...,...,...
1907,अमेरीका वनाच्छादित भाग : अमेरिका इजरायल कॉलोन...,क्या डोनाल्ड ट्रम्प नए हिटलर हैं? (वीडियो)
1520,’ [ “ ” Rep ( R ) ’ ( ) ’ “ ” “ ...,Exclusive-Ways and Means Chair Brady: Border A...
1440,मार्वल कॉमिक प्रशंसक सुनकर उत्साहित होंगे कि म...,IMAX थियेटर्स में मार्वल कॉमिक्स 'द इनहुमैन' प...
68,वॉशिंगटन - उपराष्ट्रपति माइक पेंस शुक्रवार नेश...,एंटी-अबॉर्शन रैली में पेंस बोलेंगे- द न्यूयॉर्...
